In [2]:
from kafka import KafkaProducer
import threading
import json
import random
import time

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = 'bb3a6a894c14:9093'
TOPIC_PREFIX = 'tweet-'

In [3]:
def worker(user_id: int, topic: str, num_messages: int):

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            time.sleep(2)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrumpido por el usuario.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer cerrado.")

In [ ]:
if __name__ == "__main__":
    NUM_MESSAGES = random.randint(30, 50)
    threads = []

    for user_id in range(1, 5):
        topic = f"{TOPIC_PREFIX}{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic, NUM_MESSAGES),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    print("Todos los hilos han terminado.")


[Thread-2] Iniciando, topic=tweet-2, mensajes=31
[Thread-1] Iniciando, topic=tweet-1, mensajes=31
[Thread-3] Iniciando, topic=tweet-3, mensajes=31
[Thread-4] Iniciando, topic=tweet-4, mensajes=31
[Thread-4] Sent to tweet-4: ab0cfe99-5af8-4b29-ac95-2c29845b9d15
[Thread-2] Sent to tweet-2: 0066b19d-3474-4725-af74-f335e79a053d
[Thread-3] Sent to tweet-3: 7cebbba6-9663-49d1-b3c4-60e9b0271b1b
[Thread-1] Sent to tweet-1: 5f39a92e-fee8-4467-930d-c71b014668a6
[Thread-4] Sent to tweet-4: 69c90f4d-b8ca-40ae-a5cc-41fb1dbb7304[Thread-2] Sent to tweet-2: 8d64f6d0-a790-4b6d-9054-2ffe1a2c014a
[Thread-3] Sent to tweet-3: 79ce18e8-eef0-4d4c-a781-6cf47f064c6c

[Thread-1] Sent to tweet-1: 8315560f-10da-4e11-9360-88c1207332c6
[Thread-2] Sent to tweet-2: 056a5b33-4186-4eca-89cf-f782d86141f8
[Thread-3] Sent to tweet-3: 19279de9-9133-42ee-a005-1a4600302e47
[Thread-4] Sent to tweet-4: 0a9ea531-db0b-4153-a9dd-cca77a023e1c
[Thread-1] Sent to tweet-1: cea52d2d-2219-464d-962e-9286e2555369
[Thread-2] Sent to tweet